In [98]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

# Importing Image class from PIL module 
from PIL import Image
import boto3

#returns the different labels of objects that are found in the photo
def detect_labels(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=15)

    return (response['Labels'])


#checks if a person is detected in the photo and returns the objectbox values
def getPersonObjectBox(labels):


    for label in labels:
        if label['Name'] == "Person":
            for instance in label['Instances']:
                personObjectBox = {
                    "Type": "Person",
                    "width": instance['BoundingBox']['Width'],
                    "height": instance['BoundingBox']['Height'],
                    "top": instance['BoundingBox']['Top'],
                    "left": instance['BoundingBox']['Left'],
                }
                return personObjectBox
            else:
                print('No person was found!')
                return 'No person was found!'

def cropImage(left, upper, right, lower, index, path):

    # Opens a image in RGB mode 
    im = Image.open(path)

    im_crop = im.crop((left, upper, right, lower))

    im_crop.show()

    im_crop.save(f'C:\ANPR_FOTOS\Alarm Picture\Cropped\cropped{index}.jpg', 'JPEG')


def calculateCoordinates(imageWidth, imageHeight, person):

    #calculate the coördinates of the person objectBox 

    #calculate top/left (x,y) coördinate

    x1 = imageWidth * person['left']
    y1 = imageHeight * person['top']

    #calculate bottom/right (x,y) coördinate

    x2 = x1 + imageWidth * person['width']
    y2 = y1 + imageHeight * person['height']

    #add margins to get the windshield of the car left - 40%, right + 20%, top - 15% and bottom + 15%
    
    x1 = x1 - x1 * 0.4
    y1 = y1 - y1 * 0.15

    x2 = x2 + x2 * 0.2
    y2 = y2 + y2 * 0.15

    coordinates = {
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }

    return coordinates




def main():
    # photo='Fotos-7-12-2020/20000101005741860_1_1SHP373_1_8.jpg'
    bucket='ai-essentials-anpr'

    #loop over a list of all images in the s3 bucket and crop all windshields out of the photographs

    conn = client('s3')

    #enumerate is used to keep track of the index in the loop
    for index, key in enumerate(conn.list_objects(Bucket='ai-essentials-anpr')['Contents']):
        #key['Key'] is the name of a photo in the s3 bucket
        labels=detect_labels(key['Key'], bucket)

        person = getPersonObjectBox(labels)

        #image dimensions
        imageWidth = 1920
        imageHeight = 1144

        coordinates = calculateCoordinates(imageWidth, imageHeight, person)

        print(coordinates['x1'], coordinates['y1'])
        print(coordinates['x2'], coordinates['y2'])

        cropImage(coordinates['x1'], coordinates['y1'], coordinates['x2'], coordinates['y2'], index, key['Key'])

   


if __name__ == "__main__":
    main()






TypeError: 'NoneType' object is not subscriptable

In [83]:
from boto3 import client

conn = client('s3')
for key in conn.list_objects(Bucket='ai-essentials-anpr')['Contents']:
    print(key['Key'])

101020430585_1_1SHP373_1_151.jpg
Fotos-7-12-2020/20000101020431896_1_1SHP373_1_152.jpg
Fotos-7-12-2020/20000101020433101_1_1SHP373_1_153.jpg
Fotos-7-12-2020/20000101020434235_1_1SHP373_1_154.jpg
Fotos-7-12-2020/20000101020435235_1_1SHP373_1_155.jpg
Fotos-7-12-2020/20000101020436239_1_1SHP373_1_156.jpg
Fotos-7-12-2020/20000101020437075_1_1SHP373_1_157.jpg
Fotos-7-12-2020/20000101020438332_1_1SHP373_1_158.jpg
Fotos-7-12-2020/20000101020439302_1_1SHP373_1_159.jpg
Fotos-7-12-2020/20000101020440217_1_1SHP373_1_160.jpg
Fotos-7-12-2020/20000101020441182_1_1SHP373_1_161.jpg
Fotos-7-12-2020/20000101020442041_1_1SHP373_1_162.jpg
Fotos-7-12-2020/20000101020443133_1_1SHP373_1_163.jpg
Fotos-7-12-2020/20000101020444155_1_1SHP373_1_164.jpg
Fotos-7-12-2020/20000101020445115_1_1SHP373_1_165.jpg
Fotos-7-12-2020/20000101020446297_1_1SHP373_1_166.jpg
Fotos-7-12-2020/20000101020447065_1_1SHP373_1_167.jpg
Fotos-7-12-2020/20000101020447783_1_1SHP373_1_168.jpg
Fotos-7-12-2020/20000101020448984_1_1SHP373_1_169